In [7]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [16]:
# institutionName 리스트
institution_names = [
    "DOMINION MERCHANTS AND PARTNERS LIMITED",
    "Egwafin Microfinance Bank Limited",
    # "THE DATASEED COMPANY",
    "C24 Limited",
    "Boctrust Microfinance Bank Ltd",
    "E-BARCS MICROFINANCE BANK LIMITED",
    "Tangerine Money Microfinance Bank Limited",
    "Access Bank Plc",
    "RenMoney MFB Limited",
    "PERSONAL TRUST MICROFINANCE BANK LIMITED",
    "Union Bank Nigeria Plc Lagos",
    "QUBE MICROFINANCE BANK LIMITED",
    "MIM Finance Company Limited",
    "BLUERIDGE MICROFINANCE BANK LIMITED",
    # "Mutual Trust Microfinance Bank limited",
    # "Source Microfinance Bank Ltd",
    "Access Bank Plc Lagos",
    "King Aledura Limited",
    "OCTOPUS MICROFINANCE BANK LIMITED",
    "ADDOSSER FINANCE LIMITED",
    "RenMoney MFB Nigeria Limited  Lagos",
    # "County Finance",
    "KING ALEDURA LIMITED",
    "ALTRACRED FINANCE INVESTMENT LIMITED",
    "AjoPay Financial Technology Limited",
    "Zedvance Limited Lagos",
    "Octopus Microfinance Bank Limited",
    "Balogun Fulani MFB Kwara",
    "AELLA FINANCIAL SOLUTIONS LIMITED",
    "Crednet Technologies Limited (CredPal)",
    "First Bank of Nigeria Plc",
    "Creditwise Financials Limited",
    "GOLDEN OX PARTNERS LIMITED",
    "Trives Financials company",
    # "Credit Direct Ltd Lagos",
    # "One Finance And Investments Limited",
    "Eric Vista Limited",
    "ALVANA MICROFINANCE BANK  LIMITED",
    "Addosser Microfinance Bank Limited Lagos",
    "Trafalgar Associates Limited",
    "Carbon Microfinance Bank Limited",
    "First City Monument Bank Limited",
    "Kuda Bank",
    "Blue Ridge Microfinance Bank Limited",
    "Sprint Horse Investment Limited",
    "Aella Financial Solution Ltd",
    "CoolBucks",
    "Addosser Finance Limited",
    "Page International Financial Services Limited",
    "EBARCLAYS MFB FCT",
    "Ilisan MFB Ogun",
    # "The Alternative Bank",
    "CIT Microfinance Bank Lagos",
    "Zenith Bank Plc Lagos",
    "Fairmoney Microfinance Bank Limited",
    "FIRST CITY MONUMENT BANK LIMITED",
    "ASHA MICROFINANCE BANK LIMITED",
    "Sycamore Integrated Services Limited",
    "TrustBanc Microfinance Bank Limited",
    "Alvana Microfinance Bank Limited",
    "FAIRMONEY MICROFINANCE BANK LIMITED",
    "Bravewood Finance Company Limited",
    "NOLT FINANCE COMPANY LIMITED",
    "UNITED BANK FOR AFRICA PLC",
    "CREDNET TECHNOLOGIES LIMITED",
    "UCEE Microfinance Bank Limited",
    "CashXpress Nigeria Limited"
]

# Step 1 ~ Step 3: institution_names를 하나씩 처리
for institution_name in institution_names:
    # Step 1: NULL 값이 있는지 확인
    check_query = """
    SELECT "identifiedInstitution"
    FROM credit_history
    WHERE "institutionName" = %s
    AND "identifiedInstitution" IS NULL;
    """
    cursor.execute(check_query, (institution_name,))
    null_record = cursor.fetchone()

    if null_record:  # NULL이 존재하면 처리 시작
        first_word = institution_name.split()[0]  # 첫 번째 단어 추출

        # Step 2: 첫 번째 단어를 이용해 identifiedInstitution 검색
        search_query = """
        SELECT DISTINCT "identifiedInstitution"
        FROM credit_history
        WHERE "identifiedInstitution" ILIKE %s
        LIMIT 1;
        """
        cursor.execute(search_query, (f'%{first_word}%',))
        result = cursor.fetchone()

        if result:
            identified_institution = result[0]

            # Step 3: identifiedInstitution 업데이트
            update_query = """
            UPDATE credit_history
            SET "identifiedInstitution" = %s
            WHERE "institutionName" = %s;
            """
            cursor.execute(update_query, (identified_institution, institution_name))
            print(f"Updated: {institution_name} -> {identified_institution}")

# 변경 사항 커밋
conn.commit()


Updated: Access Bank Plc -> access bank plc lagos
Updated: RenMoney MFB Limited -> renmoney mfb nigeria limited lagos
Updated: PERSONAL TRUST MICROFINANCE BANK LIMITED -> personal trust microfinance bank limited lagos
Updated: Access Bank Plc Lagos -> access bank plc lagos
Updated: RenMoney MFB Nigeria Limited  Lagos -> renmoney mfb nigeria limited lagos
Updated: Zedvance Limited Lagos -> zedvance limited
Updated: Crednet Technologies Limited (CredPal) -> crednet technologies
Updated: First Bank of Nigeria Plc -> first bank plc lagos
Updated: Eric Vista Limited -> eric vsta limited
Updated: First City Monument Bank Limited -> first bank plc lagos
Updated: Kuda Bank -> kuda microfinance bank limited
Updated: Blue Ridge Microfinance Bank Limited -> blueridge microfinance bank limited
Updated: CIT Microfinance Bank Lagos -> city gate global investment lagos
Updated: Fairmoney Microfinance Bank Limited -> fairmoney microfinance bank limited
Updated: FIRST CITY MONUMENT BANK LIMITED -> firs